# LangChain 모듈 - Agent
Agent의 핵심 아이디어는 언어 모델을 사용하여 수행할 일련의 작업을 선택하는 것입니다.  
Chain에서는 일련의 작업순서를 코드에 하드코딩 합니다.   
하지만, Agent에서는 언어모델이 추론엔진으로 사용되어, 어떤 작업을 어떤 순서로 수행할지 결정합니다.
  
- Code 출처 : https://python.langchain.com/docs/modules/agents/
- 수정사항 : 설명과 프롬프트 내용을 한글로 변경

In [3]:
%pip install -qU langchain langchain-openai langchain-community
# %pip install -qU langchain langchain-openai langchain-community==0.0.19

Note: you may need to restart the kernel to use updated packages.


### Setup: LangSmith
Agent는 사용자에게 표시되는 출력을 반환하기 전에 입력에 따라 자체 결정된 일련의 단계를 수행합니다.      
이로 인해 이러한 시스템을 디버깅하는 것이 특히 까다로워지고, 관찰 가능성이 특히 중요해집니다.    
LangSmith는 이러한 경우에 특히 유용합니다.   
LangChain으로 빌드할 때, 모든 단계는 LangSmith에서 자동으로 추적됩니다.   
LangSmith 설정은 다음 환경변수로 합니다.  
LangChain API Key :  https://smith.langchain.com/  
- export LANGCHAIN_TRACING_V2="true"  
- export LANGCHAIN_API_KEY="<your-api-key>  "

In [6]:
import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"
# os.environ["LANGCHAIN_API_KEY"] = "ls__b65e9f75a6b74a72ad48df022821e1ce"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

### Define tools

#### Tavily : https://app.tavily.com/
- LangChain에 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있도록 내장된 도구를 가지고 있습니다.
- Tavily API 키가 필요합니다.

In [7]:
# os.environ["TAVILY_API_KEY"] = "<your Tavily API key if not set as env var>"

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
search.invoke("what is the weather in SF")

[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US',
  'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detected in your area\nCold & Flu Forecast\nFlu risk is low in your area\nWe recognize our responsibility to use data and technology for good. recents\nSpecialty Forecasts\n10 Day Weather-San Francisco, CA\nToday\nMon 18 | Day\nConsiderable cloudiness. Tue 19\nTue 19 | Day\nLight rain early...then remaining cloudy with showers in the afternoon. Wed 27\nWed 27 | Day\nOvercast with rain showers at times."},
 {'url': 'https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US',
  'content': "Today's and tonight's San Francisco, CA weather forecast, weath

#### Retriever
자체 데이터를 검색하는 Retriever를 만듭니다.

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [10]:
retriever.get_relevant_documents("how to upload a dataset")[0]

Document(page_content="to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn how to create an account, obtain an API key, and configure your environment.Pricing: Learn about the pricing model for LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of LangSmith.Prompt Hub Learn about the Prompt Hub, a prompt management tool built into LangSmith.Additional Resources‚ÄãLangSmith Cookbook: A collection of tutorials and end-to-end walkthroughs using LangSmith.LangChain Python: Docs for the Python LangChain library.LangChain Python API Reference: documentation to review the core APIs of LangChain.LangChain JS: Docs for the TypeScript LangChain libraryDiscord: Join us on our Discord to discuss all things LangChain!Conta

이제 검색을 수행할 인덱스를 채웠으므로 이를 tool(agent가 적절하게 사용하는 데 필요한 형식)로 쉽게 전환할 수 있습니다.

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

#### 다운스트림에서 사용할 도구 리스트

In [12]:
tools = [search, retriever_tool]

### Agent 생성

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Agent 실행

In [17]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentü¶úÔ∏èüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookLangSmithOn this pageLangSmithIntroduction‚ÄãLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start‚ÄãTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps‚ÄãCheck out the following sections to learn more about LangSmith

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It can help with testing by providing capabilities for debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs.\n\nSome ways LangSmith can help with testing include:\n- Tracing capabilities\n- Evaluation capabilities\n- Integration with LangChain framework\n- Debugging tools\n- Monitoring features\n\nFor more detailed information on how LangSmith can specifically assist with testing, you can refer to the LangSmith documentation and user guide.'}

In [19]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco', 'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area.'}, {'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 2:27 am PDT Mar 19, 2024. Forecast Valid: 3am PDT Mar 19, 2024-6pm PDT Mar 25, 2024 .'}, {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629', 'content': 'San Francisco, CA Weather Forecast, with current conditions, wind, air q

{'input': 'whats the weather in sf?',
 'output': 'The weather in San Francisco is currently cloudy with light rain expected later in the day. You can find more detailed forecasts and current conditions on [Weather Underground](https://www.wunderground.com/forecast/us/ca/san-francisco) or [AccuWeather](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629).'}

### Adding in memory
- Agent는 Stateless로 이전 상호작용을 기억하지 못합니다.
- 메모리를 제공하려면 이전 채팅 이력을 전달해야 합니다.

In [20]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you today, Bob?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I assist you today, Bob?'}

이러한 메시지를 자동으로 추적하려면 이를 RunnableWithMessageHistory로 래핑할 수 있습니다. 

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [23]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [24]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


Your name is Bob! How can I assist you today, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob! How can I assist you today, Bob?'}